In [1]:
import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


In [2]:
import cv2


# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture(0)

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=100)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape
    

    # Extract Region of interest
    roi = frame

    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 1000:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)


            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

        
    cv2.imshow("roi", roi)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

{12: (580, 351), 13: (545, 319)}
{14: (213, 242), 15: (492, 214), 16: (379, 111), 17: (302, 116), 18: (590, 327), 19: (373, 230), 20: (594, 121)}
{14: (213, 242), 15: (492, 214), 16: (379, 111), 17: (292, 118), 18: (590, 327), 19: (373, 230), 20: (594, 121)}
{18: (567, 336), 19: (373, 230), 20: (594, 121), 16: (379, 111), 17: (292, 118)}
{18: (564, 342), 21: (401, 200)}
{18: (572, 338), 22: (400, 154)}
{24: (467, 226), 25: (400, 246), 26: (437, 177)}
{24: (467, 226), 25: (400, 246), 26: (423, 172), 27: (493, 262), 28: (400, 216)}
{24: (467, 224), 27: (493, 262), 28: (400, 216), 26: (423, 172)}
{32: (531, 163), 33: (381, 108), 34: (591, 307), 35: (475, 208), 36: (570, 159)}
{34: (592, 312), 35: (475, 208), 36: (570, 159), 33: (381, 108)}
{34: (592, 312), 35: (489, 206), 36: (570, 159), 33: (381, 108)}
{34: (592, 312), 35: (489, 206), 36: (579, 158), 33: (381, 108)}
{34: (592, 312), 35: (489, 206), 36: (579, 158), 33: (375, 110)}
{34: (592, 312), 35: (480, 204), 36: (579, 158), 33: (375,

{100: (529, 280)}
{104: (486, 321), 105: (568, 356), 106: (412, 231)}
{104: (486, 321), 105: (568, 356), 106: (412, 231)}
{104: (486, 321), 105: (568, 356), 106: (412, 231)}
{107: (453, 338), 108: (382, 412)}
{108: (372, 413), 107: (453, 338)}
{108: (359, 412), 109: (591, 334), 110: (419, 340)}
{108: (359, 412), 109: (591, 334), 110: (428, 340)}
{111: (386, 412), 112: (535, 311), 113: (374, 317)}
{111: (386, 412), 112: (535, 310), 113: (374, 317), 114: (257, 412)}
{111: (386, 412), 112: (535, 310), 113: (353, 318), 114: (257, 412)}
{111: (386, 412), 114: (257, 412), 112: (535, 310), 113: (353, 318)}
{111: (386, 412), 114: (257, 412), 112: (535, 308), 113: (353, 318)}
{111: (386, 412), 114: (257, 412), 112: (535, 308), 113: (353, 318)}
{111: (381, 412), 112: (535, 308), 113: (353, 318)}
{111: (381, 412), 112: (531, 308), 113: (353, 318)}
{111: (381, 412), 112: (531, 308), 113: (336, 316)}
{111: (379, 412), 112: (531, 308), 113: (336, 316)}
{111: (379, 412), 112: (532, 310), 113: (336, 3